In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, f1_score
from tqdm import tqdm
import numpy as np
import os

In [22]:
# Set random seed for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# 1. Data Preprocessing
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize for efficiency in FHE
    transforms.Grayscale(),       # Convert to grayscale for single-channel input
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1] for HE compatibility
])

# Load dataset
data_dir = 'C:\\Users\\Harshavardan pd\\OneDrive\\Desktop\\new crypt\\COVID-19_Radiography_Dataset'
dataset = datasets.ImageFolder(
    root=data_dir,
    transform=data_transforms,
    is_valid_file=lambda path: path.endswith(('.png', '.jpg', '.jpeg')) and 'images' in path
)

In [23]:
# 2. Programmatic Train/Validation/Test Split
indices = list(range(len(dataset)))
labels = [dataset.targets[i] for i in indices]

# First, split into train+val (80%) and test (20%)
train_val_indices, test_indices = train_test_split(
    indices, test_size=0.2, stratify=labels, random_state=42
)
# Then, split train+val into train (70% of total) and validation (10% of total)
train_indices, val_indices = train_test_split(
    train_val_indices, test_size=0.125, stratify=[labels[i] for i in train_val_indices], random_state=42
)

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler, num_workers=4, pin_memory=True)
val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler, num_workers=4, pin_memory=True)
test_loader = DataLoader(dataset, batch_size=32, sampler=test_sampler, num_workers=4, pin_memory=True)

In [24]:
# Print dataset info
print(f"Classes: {dataset.classes}")
print(f"Train set size: {len(train_indices)}, Validation set size: {len(val_indices)}, Test set size: {len(test_indices)}")

Classes: ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']
Train set size: 14815, Validation set size: 2117, Test set size: 4233


In [25]:
# 3. Define HE-Compatible CNN
class HECNN(nn.Module):
    def __init__(self):
        super(HECNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)  # AvgPool for better HE compatibility
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 4)  # 4 classes: COVID, Lung_Opacity, Normal, Viral Pneumonia

    def forward(self, x):
        x = self.conv1(x)
        x = x * x  # Square activation (HE-friendly)
        x = self.pool(x)
        x = self.conv2(x)
        x = x * x
        x = self.pool(x)
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc1(x)
        x = x * x
        x = self.fc2(x)
        return x


In [26]:
# 4. Setup Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = HECNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cuda


In [27]:
# 5. Validation Function
def validate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    return val_loss, val_accuracy

In [28]:
# 6. Training Function with Progress Bar and Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10, patience=3):
    model.train()
    best_val_loss = float('inf')
    train_loss_history = []
    val_loss_history = []
    val_accuracy_history = []
    epochs_without_improvement = 0

    for epoch in range(epochs):
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for images, labels in progress_bar:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            progress_bar.set_postfix({'batch_loss': loss.item()})

        epoch_loss = running_loss / len(train_loader)
        train_loss_history.append(epoch_loss)

        # Validate after each epoch
        val_loss, val_accuracy = validate_model(model, val_loader, criterion)
        val_loss_history.append(val_loss)
        val_accuracy_history.append(val_accuracy)

        print(f"Epoch {epoch+1}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

        # Save model if validation loss improves
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'covid_radiograph_model_best.pth')
            print(f"Model saved (improved val loss: {best_val_loss:.4f})")
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        # Early stopping
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs (no improvement in validation loss for {patience} epochs)")
            break

    return train_loss_history, val_loss_history, val_accuracy_history

In [29]:

# 7. Evaluation Function with Precision and F1 Score
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Test Precision (macro): {precision:.4f}")
    print(f"Test F1 Score (macro): {f1:.4f}")
    return accuracy, precision, f1

In [30]:

# 8. Train and Evaluate
# Change the 'epochs' parameter HERE to adjust the number of training epochs
# Set to 10 to allow early stopping to find the optimal epoch (likely around 5-7 based on previous results)
train_loss_history, val_loss_history, val_accuracy_history = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10, patience=3)
print("Train Loss History:", [f"{loss:.4f}" for loss in train_loss_history])
print("Validation Loss History:", [f"{loss:.4f}" for loss in val_loss_history])
print("Validation Accuracy History:", [f"{acc:.2f}%" for acc in val_accuracy_history])
accuracy, precision, f1 = evaluate_model(model, test_loader)


Epoch 1/10: 100%|██████████| 463/463 [00:19<00:00, 23.21it/s, batch_loss=0.377]


Epoch 1, Train Loss: 0.7214, Val Loss: 0.5516, Val Accuracy: 77.75%
Model saved (improved val loss: 0.5516)


Epoch 2/10: 100%|██████████| 463/463 [00:20<00:00, 22.98it/s, batch_loss=0.253]


Epoch 2, Train Loss: 0.5213, Val Loss: 0.4493, Val Accuracy: 81.91%
Model saved (improved val loss: 0.4493)


Epoch 3/10: 100%|██████████| 463/463 [00:20<00:00, 22.14it/s, batch_loss=0.434]


...

Epoch 8, Train Loss: 0.2717, Val Loss: 0.4307, Val Accuracy: 84.70%


Epoch 9/10: 100%|██████████| 463/463 [00:19<00:00, 23.68it/s, batch_loss=0.221] 


Epoch 9, Train Loss: 0.2363, Val Loss: 0.4206, Val Accuracy: 86.40%
Model saved (improved val loss: 0.4206)


Epoch 10/10: 100%|██████████| 463/463 [00:19<00:00, 23.62it/s, batch_loss=0.161] 


Epoch 10, Train Loss: 0.2081, Val Loss: 0.4553, Val Accuracy: 84.98%
Train Loss History: ['0.7214', '0.5213', '0.4316', '0.3831', '0.3467', '0.3104', '0.2890', '0.2717', '0.2363', '0.2081']
Validation Loss History: ['0.5516', '0.4493', '0.4434', '0.4372', '0.4578', '0.4266', '0.4526', '0.4307', '0.4206', '0.4553']
Validation Accuracy History: ['77.75%', '81.91%', '83.23%', '83.28%', '84.18%', '84.41%', '83.75%', '84.70%', '86.40%', '84.98%']
Test Accuracy: 84.72%
Test Precision (macro): 0.8636
Test F1 Score (macro): 0.8557


In [31]:
# Save final model
torch.save(model.state_dict(), 'covid_radiograph_model_final.pth')
print("Final model saved as 'covid_radiograph_model_final.pth'")

Final model saved as 'covid_radiograph_model_final.pth'
